# Parcial de Carteras 2023

Consideramos que nuestro universo empresarial está compuesto por las siguientes empresas. Trabajamos con datos diarios y la rentabilidad libre de riesgo es del 5.5,285%.

1.     ^GSPC S&P500 indice de mercado

2.     AAPL Apple Inc

3.     AIG American International Group, Inc.

4.     AMZN Amazon.com, Inc.

5.     AXP American Express

6.     BAC Bank of America

7.     CLX The Clorox Company

8.     CSCO CISCO

9.     CVX Chevron

10.   DPZ Domino's Pizza, Inc.

11.   EBAY eBay Inc.

12.   GOOGL Alphabet Inc.

13.   KO Coca Cola

14.   MSFT  Microsoft Corporation

15.   TEF Telefónica, S.A.

La cartera que define la línea del mercado de capitales sí sólo trabajamos con los 14 títulos ofrece un rentabilidad mayor que el índice de mercado (^GSPC S&P500 indice de mercado). Suponemos que durante un año se cotizan 252 días y no se puede utilizar el optimizador para contestar a esta pregunta.
La respuesta errónea resta.



In [79]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression
import math
import matplotlib.pyplot as plt
import scipy.optimize as sco
import statsmodels.api as sm

Obtenemos los precios de Yahoo Finance

In [80]:
stocks_list = 'AAPL AIG AMZN AXP BAC CLX CSCO CVX DPZ EBAY GOOGL KO MSFT TEF' #ojo no incluir el indice
start_date = '2018-08-01'
end_date = '2023-09-01'

In [81]:
data = yf.download(tickers=stocks_list, start=start_date,
                   end=end_date, interval="1d")['Adj Close'].dropna()
print("Precio de las acciones:")
symbols=data.columns
noa=len(symbols)
print(data.head())

[*********************100%***********************]  14 of 14 completed
Precio de las acciones:
                 AAPL        AIG       AMZN        AXP        BAC         CLX  \
Date                                                                            
2018-08-01  48.013962  47.416664  89.858498  92.262833  27.601383  116.324989   
2018-08-02  49.417446  47.563244  91.716499  92.522598  27.627878  123.504288   
2018-08-03  49.560429  46.261204  91.164497  93.505981  27.831028  124.059219   
2018-08-06  49.817757  46.062885  92.387497  93.626602  27.839855  124.440727   
2018-08-07  49.350727  45.692112  93.124001  94.600700  27.831028  122.671890   

                 CSCO        CVX         DPZ       EBAY      GOOGL         KO  \
Date                                                                            
2018-08-01  35.728283  99.746849  256.222076  30.668215  61.649502  39.352844   
2018-08-02  36.300140  98.450195  262.297333  30.788700  62.056499  39.174706   
2018-08-03  3

Estimamos las rentabilidades diarias

In [82]:
diario_ret = np.log(data).diff().dropna()
print("Rentabilidad Diaria:")
print(diario_ret.head())

Rentabilidad Diaria:
                AAPL       AIG      AMZN       AXP       BAC       CLX  \
Date                                                                     
2018-08-02  0.028812  0.003087  0.020466  0.002812  0.000959  0.059888   
2018-08-03  0.002889 -0.027757 -0.006037  0.010572  0.007326  0.004483   
2018-08-06  0.005179 -0.004296  0.013326  0.001289  0.000317  0.003070   
2018-08-07 -0.009419 -0.008082  0.007940  0.010350 -0.000317 -0.014316   
2018-08-08  0.000676 -0.010434  0.012825  0.007912  0.009161 -0.004392   

                CSCO       CVX       DPZ      EBAY     GOOGL        KO  \
Date                                                                     
2018-08-02  0.015879 -0.013085  0.023434  0.003921  0.006580 -0.004537   
2018-08-03  0.007029  0.002340  0.009888  0.013158 -0.002396  0.009483   
2018-08-06  0.010914  0.002013  0.002287  0.009756 -0.000396  0.000643   
2018-08-07  0.006446  0.007055 -0.009649 -0.006197  0.014574 -0.003221   
2018-08-08  0.00

Calculamos la rentabilidad media diaria

In [83]:
mean_ret= diario_ret.mean()
print (mean_ret*252)
assets_mean_ret = np.array(mean_ret)

AAPL     0.268539
AIG      0.040281
AMZN     0.084543
AXP      0.105172
BAC      0.007484
CLX      0.056442
CSCO     0.091822
CVX      0.094454
DPZ      0.080852
EBAY     0.074582
GOOGL    0.156133
KO       0.080986
MSFT     0.233291
TEF     -0.089132
dtype: float64


Calculamos la matriz de covarianzas

In [84]:
cov_mat = diario_ret.cov()
var_cov_mat = np.array(cov_mat)
print("Matriz de varianzas covarianzas:")
print(cov_mat)

Matriz de varianzas covarianzas:
           AAPL       AIG      AMZN       AXP       BAC       CLX      CSCO  \
AAPL   0.000436  0.000215  0.000307  0.000240  0.000216  0.000073  0.000225   
AIG    0.000215  0.000672  0.000147  0.000451  0.000435  0.000017  0.000219   
AMZN   0.000307  0.000147  0.000528  0.000187  0.000169  0.000046  0.000185   
AXP    0.000240  0.000451  0.000187  0.000551  0.000409  0.000017  0.000224   
BAC    0.000216  0.000435  0.000169  0.000409  0.000500  0.000019  0.000220   
CLX    0.000073  0.000017  0.000046  0.000017  0.000019  0.000288  0.000071   
CSCO   0.000225  0.000219  0.000185  0.000224  0.000220  0.000071  0.000328   
CVX    0.000174  0.000371  0.000111  0.000334  0.000322  0.000008  0.000176   
DPZ    0.000139  0.000076  0.000158  0.000080  0.000082  0.000054  0.000106   
EBAY   0.000205  0.000164  0.000225  0.000172  0.000167  0.000076  0.000156   
GOOGL  0.000290  0.000210  0.000314  0.000235  0.000208  0.000039  0.000196   
KO     0.000120  0.

Repetimos la misma operación con el indice 

In [86]:
index_list = '^GSPC'
start_date = '2018-08-01'
end_date = '2023-09-01'

In [87]:
data_i = yf.download(tickers=index_list, start=start_date,
                   end=end_date, interval="1d")['Adj Close'].dropna()
print("Evolución del Indice de Mercad:")
print(data_i.head())

[*********************100%***********************]  1 of 1 completed
Evolución del Indice de Mercad:
Date
2018-08-01    2813.360107
2018-08-02    2827.219971
2018-08-03    2840.350098
2018-08-06    2850.399902
2018-08-07    2858.449951
Name: Adj Close, dtype: float64


In [88]:
diario_ret_i = np.log(data_i).diff().dropna()
print("Rentabilidad Diaria:")
print(diario_ret_i.head())

Rentabilidad Diaria:
Date
2018-08-02    0.004914
2018-08-03    0.004633
2018-08-06    0.003532
2018-08-07    0.002820
2018-08-08   -0.000262
Name: Adj Close, dtype: float64


In [89]:
mean_ret_i= diario_ret_i.mean()
print (mean_ret_i*252)

0.09287917496697862


Estimamos la rentabilidad libre de riesgo diaria

In [91]:
rf=0.05285/252

Definimos funciones para calcular la rentabilidad de una cartera, su volatilidad y el ratio de sharpe con signo negativo.

In [92]:
def port_ret(weights):
    return np.dot(mean_ret.T *252, weights)

In [93]:
def port_vol(weights):
    return np.sqrt(np.dot(weights.T, np.dot(cov_mat * 252, weights)))

In [94]:
def min_func_sharpe(weights):
    return -(port_ret(weights)-rf) / port_vol(weights)

Indicamos que todas las ponderaciones deben de estar en tre 0 y 1. Es decir no están permitidas las posiciones cortas.

In [95]:
bnds = tuple((0, 1) for x in range(noa))
bnds

((0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1))

Indicamos que la suma de las ponderaciones debe ser igual a 1

In [96]:
cons = ({'type': 'eq', 'fun': lambda x:np.sum(x) - 1})
cons

{'type': 'eq', 'fun': <function __main__.<lambda>(x)>}

Calculamos las ponderaciones de una cartera equipondedarada.

In [97]:
eweights = np.array(noa * [1. / noa,])
eweights

array([0.07142857, 0.07142857, 0.07142857, 0.07142857, 0.07142857,
       0.07142857, 0.07142857, 0.07142857, 0.07142857, 0.07142857,
       0.07142857, 0.07142857, 0.07142857, 0.07142857])

In [98]:
min_func_sharpe(eweights)

-0.4230032005966753

Utilizamos el optimizador para encontrar la cartera M que define la línea del mercado de capitales

In [99]:
opts = sco.minimize(min_func_sharpe, eweights, method='SLSQP', bounds=bnds, constraints=cons)
opts

     fun: -0.8350450160002154
     jac: array([-0.00071687,  0.3500658 ,  0.43516421,  0.19164857,  0.46377111,
       -0.00066102,  0.20916642,  0.07977924,  0.05708096,  0.22112307,
        0.1597185 ,  0.00708573, -0.00069639,  0.60786691])
 message: 'Optimization terminated successfully'
    nfev: 105
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([6.07955089e-01, 2.16840434e-18, 8.08814821e-17, 0.00000000e+00,
       3.37186876e-16, 3.93507709e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.20780122e-16, 2.58799059e-16, 0.00000000e+00,
       3.52694140e-01, 0.00000000e+00])

In [100]:
M=opts['x'].round(3)
M

array([0.608, 0.   , 0.   , 0.   , 0.   , 0.039, 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.353, 0.   ])

In [126]:
ret_M_m=port_ret(M)

Determina la beta de la cartera anterior, utilizando para ello el índice de mercado. Indica el resultado utilizando tres decimales

In [102]:
# Convertir las ponderaciones en una serie
M = pd.Series(M)


Calculamos la rentabilidad diaria de la cartera M

In [103]:
rentabilidad_M = np.dot(diario_ret, M)
rentabilidad_M

array([ 0.02411198,  0.00347046,  0.00356234, ...,  0.01859912,
        0.0119472 , -0.00054694])

Añadimos la constante a la rentabilidad diaria del indice

In [104]:
diario_ret_i=sm.add_constant(diario_ret_i )
diario_ret_i

,const,Adj Close
Date,,
2018-08-02,1.0,0.004914
2018-08-03,1.0,0.004633
2018-08-06,1.0,0.003532
2018-08-07,1.0,0.002820
2018-08-08,1.0,-0.000262
...,...,...
2023-08-25,1.0,0.006696
2023-08-28,1.0,0.006245
2023-08-29,1.0,0.014404


Realizamos la regresión

In [105]:
capm_model = sm.OLS(rentabilidad_M,diario_ret_i).fit()
print(capm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     3881.
Date:                Sun, 12 Nov 2023   Prob (F-statistic):               0.00
Time:                        11:14:10   Log-Likelihood:                 4169.3
No. Observations:                1279   AIC:                            -8335.
Df Residuals:                    1277   BIC:                            -8324.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.000      2.102      0.0

Berkshire Hathaway, el grupo inversor de Warren Buffett, le está sacando partido a la subida de los tipos de interés. Ante la falta de oportunidades de inversión que encuentre atractivas, el grupo ha invertido 126.401 millones de dólares, en títulos del Tesoro a corto plazo que ofrecen una rentabilidad de 5,285%.

Su cartera de inversión en acciones tiene la siguiente estructura: Apple (AAPL), con 156.800 millones; Bank of America (BAC), con 28.300; American Express (AXP), con 22.600; Coca-Cola (KO), con 22.400, y Chevron (CVX), con 189.600 millones. Para estimar las ponderaciones redondea a dos decimales.

Determina la rentabilidad esperada de la cartera.

In [112]:
BH_s=([0.63,0,0,0.09,0.11,0,0,0.08,0,0,0,0,0,0])
rentabilidad_BH_s = np.dot(diario_ret, BH_s)
ret_BH_S_m=rentabilidad_M.mean()
w=0.66
ret_BH=w*ret_BH_S_m+(1-w)*rf
print(ret_BH*252)

0.14140523268203675


¿La cartera de Berkshire Hathaway es eficiente?

Calculamos la volatilidad de la cartera BH_s accciones y luego la resultante tras incorporar el activo libre de riesgo.

In [118]:
var_BH_s=np.dot(BH_s, np.dot(cov_mat , BH_s))
var_BH=(w**2*var_BH_s)*252
vol_BH=np.sqrt(var_BH)
vol_BH

0.1753133478639716

Determinamos la ponderación que la cartera M debería tener para que la combinación con el activo libre de riesgo nos genere una volatilidad equivalente a la catera BH.

$$\begin{aligned}
0.175 &=w_M*\sigma(M)\\
w_M &=\frac{0.175}{\sigma(M)}
\end{aligned}$$

In [139]:
wM=vol_BH/port_vol(M)
wM

0.591218274836242

Calculamos la rentabilidad de la cartera que combina el 59% de la cartera M y el activo libre de riesgo

In [140]:
R=wM*port_ret(M)+(1-wM)*rf*252
R

0.16812263684484147